In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from os import path as osp
import json
import pickle

from PIL import Image


In [3]:
import sys

sys.path.insert(0, '../NSCL-PyTorch-Release/')
sys.path.insert(0, 'my_imp/')

In [45]:
from my_imp.data import gen_image_transform, gen_bbox_transform
from my_imp.copied.scene_annotation import annotate_objects
from my_imp.copied.program_translator import clevr_to_nsclseq, nsclseq_to_nsclqsseq
import nltk


from vocab import Vocab

In [17]:
from torch.utils.data import Dataset

In [32]:
img_size = 256

ans_dict_json = 'data/clevr_sample_mod/CLEVR_built_dictionaries.json'

train_ds_root = 'data/clevr_sample_mod/train/'
image_root = 'data/clevr_sample_mod/train/images/'
vocab_json = 'data/clevr_sample_mod/train/vocab.json'
train_scenes_json = 'data/clevr_sample_mod/train/scenes.json'
train_questions_json = 'data/clevr_sample_mod/train/questions.json'

val_ds_root = 'data/clevr_sample_mod/val/'
val_image_root = 'data/clevr_sample_mod/val/images/'
val_vocab_json = 'data/clevr_sample_mod/val/vocab.json'
val_scenes_json = 'data/clevr_sample_mod/val/scenes.json'
val_questions_json = 'data/clevr_sample_mod/val/questions.json'

dataset_args = {
    'train': {
        'scenes_json': train_scenes_json,
        'questions_json': train_questions_json,
        'image_root': image_root,
        'vocab_json': vocab_json,
    },
    'val': {
        'scenes_json': val_scenes_json,
        'questions_json': val_questions_json,
        'image_root': val_image_root,
        'vocab_json': val_vocab_json,
    },
}

image_transform = gen_image_transform(img_size)
bbox_transform = gen_bbox_transform(img_size)

In [54]:
class DatasetV2(Dataset):
    def __init__(self,
                 image_root,
                 vocab_json,
                 ans_dict_json,
                 bbox_transform,
                 scenes_json='',
                 questions_json='',
                 ds_root=None,
                 image_transform=None,
                ):
        super().__init__()

        self.image_root = image_root
        self.image_transform = image_transform
        self.bbox_transform = bbox_transform

        print('Loading vocab from: "{}".'.format(vocab_json))
        self.vocab = Vocab.from_json(vocab_json)
        print('Loading answers from: "{}"'.format(ans_dict_json))
        self.ans = Vocab.from_json(ans_dict_json)

        cached_scenes = osp.join(
            ds_root, 'scenes_cache.pkl') if ds_root else ''
        if cached_scenes and osp.exists(cached_scenes):
            print('==> using cached scenes: {}'.format(cached_scenes))
            with open(cached_scenes, 'rb') as f:
                self.scenes = pickle.load(f)
        else:
            with open(scenes_json, 'r') as f:
                self.scenes = json.load(f)
            self.prepare_scenes()
            with open(cached_scenes, 'wb') as f:
                pickle.dump(self.scenes, f)

        cached_questions = osp.join(
            ds_root, 'questions_cache.pkl') if ds_root else ''
        if cached_questions and osp.exists(cached_questions):
            print('==> using cached questions: {}'.format(cached_questions))
            with open(cached_questions, 'rb') as f:
                self.questions = pickle.load(f)
        else:
            with open(questions_json, 'r') as f:
                self.questions = json.load(f)
            self.prepare_questions()
            with open(cached_questions, 'wb') as f:
                pickle.dump(self.questions, f)

    def prepare_scenes(self):
        print('Preparing scenes')
        if type(self.scenes) is list:
            dummy_fp = osp.join(
                self.image_root, self.scenes[0]['image_filename'])
            scene_iter = self.scenes
        elif type(self.scenes) is dict:
            self.scenes = {int(key): val for key, val in self.scenes.items()}
            dummy_fp = osp.join(self.image_root, list(
                self.scenes.values())[0]['image_filename'])
            scene_iter = self.scenes.values()
        else:
            raise Exception(
                f"Scenes type is '{type(self.scenes)}', expeceted 'list' or 'dict'")
        dummy_image = Image.open(dummy_fp).convert('RGB')
        for i, scene in enumerate(scene_iter):
            # scene = scenes['scenes'][i]
            print(f'\r{i + 1}/{len(self.scenes)}', end='')
            objects = annotate_objects(scene)['objects']
            # scene['objects_raw'] = scene['objects']
            scene['objects'] = self.bbox_transform(dummy_image, objects)
            scene['scene_size'] = len(scene['objects'])
            scene['image_filename'] = osp.join(
                self.image_root, scene['image_filename'])

            del scene['relationships']
            del scene['objects_detection']
            del scene['directions']
        print()

    def prepare_questions(self):
        print('\nPreparing questions')
        for i, q in enumerate(self.questions):
            print(f'\r{i + 1}/{len(self.questions)}', end='')
            q['program_size'] = len(q['program'])
            q['question_raw'] = q['question']
            q['question'] = np.array(self.vocab.map_sequence(nltk.word_tokenize(q['question'].lower())), dtype='int32')
            q['answer_raw'] = q['answer']
            q['answer'] = self.ans.word2idx[q['answer']]
            # q['program_raw'] = q['program']
            # q['program_seq'] = clevr_to_nsclseq(q['program'])
            program_seq = clevr_to_nsclseq(q['program'])
            q['program_qsseq'] = nsclseq_to_nsclqsseq(program_seq)
            q['question_type'] = program_seq[-1]['op']

            del q['program']
            del q['image_filename']

        print()
        
        
    def __getitem__(self, index):
        # fd = self.questions[index]
        # scene = self.scenes[fd['image_index']]

        # image = Image.open(scene['image_filename']).convert('RGB')
        # image = self.image_transform(image)

        # Testing without vars because of memory leaks
        return {
            'image': self.image_transform(Image.open(self.scenes[self.questions[index]['image_index']]['image_filename']).convert('RGB')),
            **self.questions[index],
            'objects': self.scenes[self.questions[index]['image_index']]['objects'],
            }

    def _get_metainfo(self, index):
        return {
            'question': self.questions[index],
            'scene': self.scenes[self.questions[index]['image_index']],
        }
    
    def __len__(self):
        return len(self.questions)
        
ds = DatasetV2(
    image_root=image_root,
    vocab_json=vocab_json,
    image_transform=image_transform,
    bbox_transform=bbox_transform,
    ds_root=train_ds_root,
    scenes_json=train_scenes_json,
    ans_dict_json=ans_dict_json,
    questions_json=train_questions_json
)

Loading vocab from: "data/clevr_sample_mod/train/vocab.json".
Loading answers from: "data/clevr_sample_mod/CLEVR_built_dictionaries.json"
==> using cached scenes: data/clevr_sample_mod/train/scenes_cache.pkl
==> using cached questions: data/clevr_sample_mod/train/questions_cache.pkl


In [56]:
from my_imp.train import build_curriculum

In [71]:
subset = build_curriculum(ds, 6)
len(subset)

1608

215